In [170]:
import datetime
import streamlit as st
from PIL import Image
import numpy as np
import pandas as pd

In [224]:
file = pd.read_csv('msteams.csv', sep=',', encoding='utf-8')
#datetime(year, month, day, hour, minute, second, microsecond)
start_time, end_time = datetime.datetime(2020, 9, 4, 8, 50, 0), datetime.datetime(2020, 9, 4, 10, 0, 0)
print(end_time - start_time)
print(start_time, end_time)
file.Timestamp = pd.to_datetime(file['Timestamp'], infer_datetime_format=True)
file

1:10:00
2020-09-04 08:50:00 2020-09-04 10:00:00


,Full Name,User Action,Timestamp
0,Willie James,Joined,2020-09-03 08:57:06
1,Dwight Hernandez,Joined before,2020-09-03 08:57:06
2,Johnni White,Joined before,2020-09-03 08:57:06
3,Leslie Harvey,Joined,2020-09-03 08:57:10
4,Chad Holt,Joined,2020-09-03 08:57:14
...,...,...,...
121,Alex Wallace,Joined,2020-09-03 09:10:40
122,Alex Wallace,Left,2020-09-03 10:00:59
123,Kenneth Parker,Joined,2020-09-03 09:14:44
124,Marian Perez,Joined,2020-09-03 09:36:47


In [225]:
max(file.Timestamp.iloc[4], file.Timestamp.iloc[5])

Timestamp('2020-09-03 08:58:28')

In [226]:
df = pd.DataFrame(columns = ["Full Name", "Total Time"])
df["Full Name"] = file["Full Name"]
names = df["Full Name"].unique().tolist()
#df1 = file[file.duplicated('Full Name', keep=False)].sort_values('Timestamp')
df.sort_values(by=['Full Name']).reset_index()
df

,Full Name,Total Time
0,Willie James,NaN
1,Dwight Hernandez,NaN
2,Johnni White,NaN
3,Leslie Harvey,NaN
4,Chad Holt,NaN
...,...,...
121,Alex Wallace,NaN
122,Alex Wallace,NaN
123,Kenneth Parker,NaN
124,Marian Perez,NaN


In [227]:
for i in df.index:
    stud_join_time, stud_left_time = start_time, end_time
    time_attended = end_time - end_time
    join_flag, leave_flag = False, False
    for j in file.index:
        if df['Full Name'][j] == file['Full Name'][i]:
            if file['User Action'][j] == 'Joined before':
                first_join_time = start_time
                stud_join_time = start_time
                time_attended = end_time - stud_join_time
            elif file['User Action'][j] == 'Left':
                stud_left_time = file['Timestamp'][j]
                time_attended -= (end_time - stud_left_time)
            elif file['User Action'][j] == 'Joined':
                stud_join_time = file['Timestamp'][j]
                '''
                if not join_flag:
                    first_join_time = stud_join_time if first_join_time != start_time else first_join_time
                    join_flag = True
                '''
                time_attended += (end_time - stud_join_time)
    df["Total Time"][i] = time_attended

df['Total Time'] = pd.to_timedelta(df['Total Time'])
df['Total Time'] = df['Total Time'] - pd.to_timedelta(df['Total Time'].dt.days, unit='d')
df = df.drop_duplicates()
df.head(126)

,Full Name,Total Time
0,Willie James,01:02:54
1,Dwight Hernandez,01:10:00
2,Johnni White,01:10:00
3,Leslie Harvey,01:02:50
4,Chad Holt,01:01:01
7,Jean Rodriquez,01:02:40
8,Luis Jacobs,01:02:30
9,Marc Washington,01:02:29
10,Mattie Martin,01:01:49
11,Bonnie Elliott,01:02:21


In [ ]:
def import_data(file):
    file = pd.read_csv(file, header=["Full Name", "User Action", "Timestamp"])
    df = pd.DataFrame(columns = ["Full Name", "First Join Time", "Last Leave Time", "Total Time"])
    lists = [[] for _ in range(4)]
    
    try:
        for message in file:
                if re.search(r"[\d]{1,2}/[\d]{1,2}/[\d]{2}, [\d]{1,2}:[\d]{1,2} ", message):
                    date_time, sender_message = message.split("-")[0].strip(),' '.join(message.split("-")[1:])
                    date, time = date_time[:8].strip(), date_time[date_time.index(",")+2:].strip()
                    if isAuthor(sender_message):
                        sender, chat_message = sender_message[:sender_message.index(":")].strip(), sender_message[sender_message.index(":")+2:].strip()
                        if sender not in senders:
                            senders.append(sender)
                    else:
                        sender, chat_message = None, sender_message.strip()

                    lists[0].append(date)
                    lists[1].append(time)
                    lists[2].append(sender)
                    lists[3].append(chat_message)
                else:
                    lists[3][-1] += ' '+ message

    except Exception as e:
        print(e)

    for k, col in enumerate(df.columns):
        df[col] = lists[k]

    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.dropna()
    df["emoji"] = df.Message.apply(emoji_count)
    URLPATTERN = r'(https?://\S+)'
    df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
    return df
